In [2]:
import numpy as np
import pandas as pd
import math

np.random.seed(69)

In [3]:
# our sigmoid function, tanh is a little nicer than the standard 1/(1+e^-x)
def sigmoid(x):
    return math.tanh(x)

# derivative of our sigmoid function, in terms of the output (i.e. y)
def dsigmoid(y):
    return 1.0 - y**2

In [21]:
class neuralNet:
    bias_output = 1.0
    bias_hidden = 5.0
    # bias = np.random.rand()
    lr = 0.1
    times_with_different_lrs = 1
    training_iterations = 500
    number_of_folds = 10
    hidden_layers = 1
    hidden_nodes = [5]

    def __init__(self, filename):
        self.filename = filename
        self.data = np.asarray(pd.read_csv(filename))
        self.classes = set()
        for _ in self.data:
            self.classes.add(_[-1])
        self.classes = dict(zip(self.classes, range(len(self.classes))))
        np.random.shuffle(self.data)
        self.test_data_size = len(self.data)/self.number_of_folds
        
        self.features = len(self.data[0])-1
        self.i = self.features + 1 #bias
        #self.wi = np.asarray([np.random.rand(self.hidden_nodes) for each in xrange(self.features)])
        #self.wh = np.random.rand(self.hidden_nodes)
        
        # bias weight is always 1. only bias is added
        self.wi = np.asarray(
        [
            np.asarray([1.0/(self.features*self.hidden_nodes[0] + self.bias_hidden)]*self.hidden_nodes[0]) for _ in xrange(self.features)
        ])
        # 0th index is for first hidden node
        self.wh = np.asarray(
        [
            np.asarray([1.0/(self.hidden_nodes[0] + self.bias_output)]*self.hidden_nodes[0])
        ])
        
    def splitDataset(self, fold):
        testStart = fold*self.test_data_size
        testEnd = (fold+1)*self.test_data_size

        test = data[testStart:testEnd]
        train = np.concatenate((data[:testStart], data[testEnd:]), axis = 0)

        return np.asarray(train), np.asarray(test)
        
    def train(self):
        pass
    
    def test(self):
        pass
        
x = neuralNet('datasets/IRIS.csv')

In [5]:
def train(data, lr, weights, bias, classes, shouldBiasChange, iters=1):
    for ___ in xrange(iters):
        for each in data:
            predicted_val = np.dot(each[:-1], weights) + bias
#             print predicted_val, each[:-1], weights
            predicted_val = int(np.clip(np.sign(predicted_val),0,1))
            actual_val = classes[each[-1]]
#             print predicted_val, '\t',actual_val
            error = predicted_val - actual_val
            delW = lr * error * each[:-1]
#             print error, delW
            weights = weights - delW
            if shouldBiasChange:
                bias = bias - lr*error
    return weights, bias

def training(lr, train_data, bias_hidden, bias_output, weight_ip_hd, weight_hd_op, iters=500):
    for y in range(iters):
        error_hd = []
        for row in train_data:
            # First find the dot product of each row of the weight with the input.
            layer_1 = np.array(np.sum(row[:-1] * weight_ip_hd, axis=1) + bias_hidden, dtype=np.float64)
            layer_1 = np.apply_along_axis(sigmoid,0,layer_1)
            sig_vals = np.copy(layer_1)
            # Now for the final layer
            val_op = np.dot(layer_1,weight_hd_op) + bias_output
            output = sigmoid(val_op)
            # Now get the actual output for the row
            actual_output = row[-1]
            output_error = output * (1 - output) * (actual_output - output)
            bias_output = bias_output + (lr * output_error)
            # Backprop
            error_hd = np.multiply(np.multiply(sig_vals, (1 - sig_vals)), (weight_hd_op * output_error))
            weight_hd_op = weight_hd_op + (lr * sig_vals * output_error)
            bias_hidden = bias_hidden + (lr * error_hd)
            
            # Finished with one, another one begins
            error_hd = error_hd.reshape(1,len(bias_hidden))
            weight_ip_hd = weight_ip_hd + (lr * (error_hd.T * row[:-1]))

    return np.asarray(weight_ip_hd), np.asarray(weight_hd_op), np.asarray(bias_hidden), np.asarray(bias_output)
            

In [6]:
def test(data, weights, bias, classes):
    tp = tn = fp = fn = 0
    
    for each in data:
        predicted_val = np.dot(each[:-1], weights) + bias
        predicted_val = int(np.clip(np.sign(predicted_val),0,1))

        if (predicted_val == classes[each[-1]]):
            if predicted_val == 1:
                tp += 1
            else:
                tn += 1
        else:
            if predicted_val == 1:
                fp += 1
            else:
                fn += 1
    
    accuracy = (tp + tn)*100/(tp + tn + fp + fn)
    try:
        precision = (tp)*100/(tp + fp)
    except ZeroDivisionError:
        precision = 0
    try:
        recall = (tp)*100/(tp + fn)
    except ZeroDivisionError:
        recall = 0
    return (accuracy, precision, recall)

def testing(test_data, bias_hidden, bias_output, weight_ip_hd, weight_hd_op, threshold):
    tp = tn = fp = fn = 0.0
    for row in test_data:
        layer_1 = np.array(np.sum(row[:-1] * weight_ip_hd, axis=1) + bias_hidden, dtype=np.float64)
        layer_1 = np.apply_along_axis(sigmoid,0,layer_1)
        # Now for the final layer
        val_op = np.dot(layer_1,weight_hd_op) + bias_output
        output = sigmoid(val_op)
        
        if output >= threshold:
            pred_output = 1
        else:
            pred_output = 0
        
        actual_output = row[-1]
        
        print pred_output, actual_output, output
        
        if (pred_output == actual_output):
            if pred_output == 1.0:
                tp += 1
            else:
                tn += 1
        else:
            if pred_output == 1.0:
                fp += 1
            else:
                fn += 1

    return tp, tn, fp, fn

In [ ]:
### Parameters

filename = 'datasets/IRIS.csv'
# filename = 'datasets/SPECTF_New.csv'
bias = 1.0
shouldBiasChange = False
learning_rate = 0.1
times_with_different_lrs = 1
training_iterations = 500
number_of_folds = 10

In [ ]:
data, classes = load_data(filename)
lr = learning_rate
test_data_size = len(data)/number_of_folds

features = data.shape[1] - 1
# weights = np.random.rand(features)
weights = np.asarray([1.0/(features+1)]*features)

maxAccuracy = -float('inf')
maxAccuracyLr = 0

In [ ]:
for _ in range(times_with_different_lrs):
    totalAccuracy = totalPrecision = totalRecall = 0
    
    print 'For learning rate %.1lf\n' %lr
    weights = np.asarray([1.0/(features+1)]*features)
    
    for eachFold in range(number_of_folds):
        trainSet, testSet = splitDataset(data, eachFold, test_data_size)
        weights, bias = train(trainSet, lr, weights, bias, classes, shouldBiasChange, training_iterations)
        accuracy, precision, recall = test(testSet, weights, bias, classes)
        
        print 'Fold %d\tAccuracy : %lf\tPrecision : %lf\tRecall : %lf' %(eachFold+1, accuracy, precision, recall)

        totalAccuracy += accuracy
        totalPrecision += precision
        totalRecall += recall
    
    totalAccuracy /= float(number_of_folds)
    totalPrecision /= float(number_of_folds)
    totalRecall /= float(number_of_folds)
    
    print """
Accuracy : %lf\tPrecision : %lf\tRecall : %lf
    """ %(totalAccuracy, totalPrecision, totalRecall)
    
    if totalAccuracy > maxAccuracy:
        maxAccuracy = totalAccuracy
        maxAccuracyLr = lr
#     lr += 0.1
    print '='*100
print 'Max Accuracy is %lf for learning rate %0.1lf' %(maxAccuracy, maxAccuracyLr)

In [7]:
data = load('IRIS.csv')
# Variables
print classes
features = len(data[0]) - 1
data_size = len(data)
learning_rate = 0.1
hidden_layer_nodes = 5
threshold = 0.65

bias_at_hidden = 5
bias_at_output = 1
bias_hidden = np.array([bias_at_hidden] * hidden_layer_nodes)

wt_ip_hd = [[1.0/(features * hidden_layer_nodes + bias_at_hidden)] * features] * hidden_layer_nodes
wt_ip_hd = np.asarray(wt_ip_hd)

wt_hd_op = [1.0/(hidden_layer_nodes + bias_at_hidden)] * hidden_layer_nodes
wt_hd_op = np.asarray(wt_hd_op)

#Split the training and testing
accuracy = precision = recall = 0
for i in range(10): #1 folds
    train, test = split(data, i)
    
    wt_ip_hd, wt_hd_op, bias_hidden, bias_at_output = training(
        learning_rate,
        train,
        bias_hidden,
        bias_at_output,
        wt_ip_hd,
        wt_hd_op,
        iters=500
    )
    
    tp, tn, fp, fn = testing(test, bias_hidden, bias_at_output, wt_ip_hd, wt_hd_op, threshold)
    try:
        pre = tp / (tp + fp)
    except ZeroDivisionError:
        pre = 0
    try:
        rec = tp / (tp + fn)
    except ZeroDivisionError:
        rec = 0
        
    acc = (tp + tn) / (tp + tn + fp + fn)
    print acc, pre, rec
    
    accuracy += acc
    precision += pre
    recall += rec
    
accuracy /= 10
precision /= 10
recall /= 10

print "Accuracy = {}, Precision = {}, Recall = {}".format(accuracy, precision, recall)

{0: 0, 1: 1}
1 1 0.844738273177
1 1 0.890878674239
0 0 0.288553339947
1 1 0.672346140652
1 0 0.883230627713
1 1 0.733791478999
0 0 0.284276686554
1 1 0.890879365533
0 0 0.284138971949
0 0 0.286945129967
0 1 0.286516176942
0 0 0.286173165305
1 1 0.890878906811
1 1 0.890879358097
0 0 0.284198114006
1 1 0.890879364473
0 0 0.295779616514
1 1 0.852673868031
0.888888888889 0.9 0.9
0 1 0.31357146809
0 0 0.24868772122
0 0 0.242049341304
0 0 0.242377137371
1 1 0.945561836355
1 1 0.945695783032
0 1 0.304776743587
0 1 0.257753605117
1 1 0.945454109106
0 0 0.25965832328
0 0 0.242168957078
1 1 0.945695783089
1 1 0.945693540916
0 1 0.243929864316
0 1 0.244609431498
1 1 0.945619226899
1 1 0.943986405319
1 1 0.945695783101
0.722222222222 1.0 0.615384615385
0 0 0.238669083071
0 0 0.228056983623
0 0 0.228619347352
1 1 0.960971515725
0 0 0.228147884928
1 1 0.957713333097
0 1 0.228100715935
0 1 0.238780227882
0 0 0.228057018346
0 1 0.228056975689
0 1 0.228056984084
1 1 0.960970764162
0 0 0.247214209817
0 